In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')

In [54]:
df = pd.read_csv('./ratings_Beauty.csv')
df.head()
df.shape
df.dtypes
df.info()
df.describe()['Rating'].T
print('Minimum rating is: %d' %(df.Rating.min()))
print('Maximum rating is: %d' %(df.Rating.max()))


,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,205616461,5,1369699200
1,A3JM6GV9MNOF9X,558925278,3,1355443200
2,A1Z513UWSAAO0F,558925278,5,1404691200
3,A1WMRR494NWEWV,733001998,4,1382572800
4,A3IAAVS479H7M7,737104473,1,1274227200


(1048575, 4)

UserId       object
ProductId    object
Rating        int64
Timestamp     int64
dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   UserId     1048575 non-null  object
 1   ProductId  1048575 non-null  object
 2   Rating     1048575 non-null  int64 
 3   Timestamp  1048575 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 32.0+ MB


count    1.048575e+06
mean     4.176864e+00
std      1.306543e+00
min      1.000000e+00
25%      4.000000e+00
50%      5.000000e+00
75%      5.000000e+00
max      5.000000e+00
Name: Rating, dtype: float64

Minimum rating is: 1
Maximum rating is: 5


## Handling Missing values

In [55]:
print('Number of missing values across columns: \n',df.isnull().sum())

Number of missing values across columns: 
 UserId       0
ProductId    0
Rating       0
Timestamp    0
dtype: int64


## Unique Users and products

In [56]:
print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",df.shape[0])
print("Total No of Users   :", len(np.unique(df.UserId)))
print("Total No of products  :", len(np.unique(df.ProductId)))

Total data 
--------------------------------------------------

Total no of ratings : 1048575
Total No of Users   : 736653
Total No of products  : 97987


In [57]:
df.drop(['Timestamp'], axis=1,inplace=True)
df.head()

,UserId,ProductId,Rating
0,A39HTATAQ9V7YF,205616461,5
1,A3JM6GV9MNOF9X,558925278,3
2,A1Z513UWSAAO0F,558925278,5
3,A1WMRR494NWEWV,733001998,4
4,A3IAAVS479H7M7,737104473,1


## Analyzing the rating

In [58]:
no_of_rated_products_per_user = df.groupby(by='UserId')['Rating'].count().sort_values(ascending=False)

no_of_rated_products_per_user.head()
no_of_rated_products_per_user.describe()
quantiles = no_of_rated_products_per_user.quantile(np.arange(0,1.01,0.01), interpolation='higher')
print('\n No of rated product more than 50 per user : {}\n'.format(sum(no_of_rated_products_per_user >= 50)) )

UserId
A3M174IC0VXOS2    278
A281NPSIMI1C2R    245
A1RRMZKOMZ2M7J    200
A3KEZLJ59C1JVH    178
A2V5R832QCSOMX    173
Name: Rating, dtype: int64

count    736653.000000
mean          1.423431
std           1.598452
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         278.000000
Name: Rating, dtype: float64


 No of rated product more than 50 per user : 58



## Popularity Based Recommendation

In [80]:
new_df=df.groupby("ProductId").filter(lambda x:x['Rating'].count() >=50)
new_df.head()

,UserId,ProductId,Rating
1705,A2SR9M2CWC2OCP,9790790961,4
1706,A3V1EVBYP0U77W,9790790961,2
1707,AVJKKAS4P52P9,9790790961,5
1708,A2NQQDBBGFW1OT,9790790961,5
1709,A1OFNEUHZ7BSCB,9790790961,5


In [60]:
new_df.groupby('ProductId')['Rating'].mean().head()

ProductId
9790790961    4.271429
9790799829    2.640000
B00004TMFE    3.445255
B00004TUBL    4.634409
B00004TUBV    4.269006
Name: Rating, dtype: float64

In [61]:
new_df.groupby('ProductId')['Rating'].mean().sort_values(ascending=False).head()

ProductId
B002YFN49I    4.948980
B001PX1AIC    4.920635
B000YT5NIG    4.873684
B000G686X6    4.871429
B000052YN6    4.868852
Name: Rating, dtype: float64

In [62]:
new_df.groupby('ProductId')['Rating'].count().sort_values(ascending=False).head()

ProductId
B001MA0QY2    7533
B0009V1YR8    2869
B0000YUXI0    2143
B000ZMBSPE    2041
B003BQ6QXK    1918
Name: Rating, dtype: int64

In [81]:
ratings_mean_count = pd.DataFrame(new_df.groupby('ProductId')['Rating'].mean())
ratings_mean_count.head()

,Rating
ProductId,
9790790961,4.271429
9790799829,2.640000
B00004TMFE,3.445255
B00004TUBL,4.634409
B00004TUBV,4.269006


In [64]:
ratings_mean_count['rating_counts'] = pd.DataFrame(new_df.groupby('ProductId')['Rating'].count())
ratings_mean_count.head()
ratings_mean_count['rating_counts'].max()

,Rating,rating_counts
ProductId,,
9790790961,4.271429,70
9790799829,2.640000,50
B00004TMFE,3.445255,137
B00004TUBL,4.634409,558
B00004TUBV,4.269006,171


7533

In [67]:
popular_products = pd.DataFrame(new_df.groupby('ProductId')['Rating'].count())
most_popular = popular_products.sort_values('Rating', ascending=False)
most_popular

,Rating
ProductId,
B001MA0QY2,7533
B0009V1YR8,2869
B0000YUXI0,2143
B000ZMBSPE,2041
B003BQ6QXK,1918
...,...
B00132CL3S,50
B000P20UCG,50
B000P24EI2,50


## Collaberative filtering (Item-Item recommedation)

In [69]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

In [70]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_df,reader)

In [71]:
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [72]:
# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [79]:
# run the trained model against the testset
test_pred = algo.test(testset)
test_pred

[Prediction(uid='AJ1AFWRHSUWLN', iid='B0009V1YR8', r_ui=2.0, est=4.171397904054479, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='AAJQ6TBRDZXYD', iid='B001PNFFLK', r_ui=5.0, est=4.171397904054479, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A1DOCJTU77JSHM', iid='B00138Y8NI', r_ui=5.0, est=4.171397904054479, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A21PAULUIG20M8', iid='B000JQQHSE', r_ui=5.0, est=4.171397904054479, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A2JOGP2Q8BK5IX', iid='B002BFX6EO', r_ui=4.0, est=4.171397904054479, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='ATUXG4KRH0NUJ', iid='B0009V1YR8', r_ui=1.0, est=4.171397904054479, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A2JGRWYMFAORAP

In [75]:
# get RMSE
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Item-based Model : Test Set
RMSE: 1.2962


1.2961539063549508